# literature generator
generat lit review from a json blob

In [1]:
import os
import openai
from openai import OpenAI
openai.organization = "org-raWgaVqCbuR9YlP1CIjclYHk" # Harvard
openai.api_key = os.getenv("OPENAI_API_KEY")

print(True if openai.api_key else False)

True


In [5]:
def create_url(id, is_doi=False, full_details=True):
    endpoint = "https://api.semanticscholar.org/graph/v1/paper/"
    fields = '?fields=title,abstract,url,year,authors,referenceCount,citationCount'
    if full_details:
        fields += ',citations,references'
    
    if is_doi:
        return endpoint + "DOI:" + id + fields
    else: 
        return endpoint + id + fields

In [6]:
import requests
import time

# calling the semantic scholar api
def api_call(api_endpoint):
    response = requests.get(api_endpoint)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(api_endpoint)
        return None

# a lot of error handling, hasn't broken so far
def try_api_call(api_endpoint):
    try:
        response = requests.get(api_endpoint)
        if response.status_code == 429:
            print("Rate limit reached. Waiting before retrying...")
            time.sleep(20)  # Sleep for a sec
            return api_call(api_endpoint)  # Retry the request
        response.raise_for_status()  # Will raise an HTTPError for other bad status codes
        return response.json()
        ## code to keep only the first 3 authors
        # if 'authors' in data and len(data['authors']) > 3:
        #         data['authors'] = data['authors'][:3]  
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - {api_endpoint}")
        return None
    except Exception as err:
        print(f"An error occurred: {err} - {api_endpoint}")
        return None


In [7]:
# prepare information from json blob
import json

# Read the JSON data from file
filename = './seminal/small.json'
with open(filename, 'r') as file:
    papers = json.load(file)

print("number of papers: ", len(papers))

basics = {}

for id, info in papers.items():
    paper_info = try_api_call(create_url(id))
    basics[paper_info['paperId']] = {
        'title': paper_info['title'],
        'abstract': paper_info['abstract'],
        'year': paper_info['year'],
        'author': paper_info['authors'][0]['name'] if paper_info['authors'][0] else None
    }

print(basics.keys())

number of papers:  9
dict_keys(['5642a582b9a9a238e0211ca85e796e9238cd6057', '257cb4317665951b77aa52d4510c31c06c11de87', '40a41644bf3682f9153b717e898c0e5364363618', 'b4fdbc2ba5d4278f39538cc1ad87048c3e2b84c6', '612de74da35f0270f7c56267c7313483df78ac6a', 'adf93e726f33066c6d9ac53332274b287878786d', '0a737f9ccd28a2409471388fedac2c07e95ae5c9', 'e3004293f727e6cfaa0b5e3cd918efbf4310a9f6', '5aad2cf82e1717ab101786f14571bb318399da64'])


In [9]:
paper_abstracts = ""

for id, entry in basics.items():
    if not entry['abstract']:
        continue
    paper_abstracts += "Title: " + entry["title"] + '\n' + "Abstract: " + entry["abstract"] + '\n' + "Author and year: (" + entry["author"] + ', ' + str(entry["year"]) + ')\n\n'

print("final string")
print(paper_abstracts)

final string
Title: A Global Model for Forecasting Political Instability
Abstract: Examining onsets of political instability in countries worldwide from 1955 to 2003, we develop a model that distinguishes countries that experienced instability from those that remained stable with a two-year lead time and over 80% accuracy. Intriguingly, the model uses few variables and a simple specification. The model is accurate in forecasting the onsets of both violent civil wars and nonviolent democratic reversals, suggesting common factors in both types of change. Whereas regime type is typically measured using linear or binary indicators of democracy/autocracy derived from the 21-point Polity scale, the model uses a nonlinear five-category measure of regime type based on the Polity components. This new measure of regime type emerges as the most powerful predictor of instability onsets, leading us to conclude that political institutions, properly specified, and not economic conditions, demography,

In [10]:
client = OpenAI()

with open('res.txt', 'w') as txtfile:
    completion = client.chat.completions.create(
        # turbo (1106-preview) has 128k context window, about 300 pages of text
        model="gpt-4-1106-preview", # test with: gpt-3.5-turbo, run final: gpt-4-1106-preview
        messages=[
            {"role": "system", "content": 
                """You are a writing a literature review to give an overview of the methodologies and history of 
                modeling, predicting, and forecasting armed civil conflicts. I will provide you with multiple existing research 
                papers on the subject with the paper title, abstract, authors, and year of publication.
                Please provide a detailed literature review based on these papers, focusing on the 
                methodologies used, the evolution of these techniques over time, and their effectiveness 
                in conflict analysis. Please include references in the text in an (author, year) format."""},
            {"role": "user", "content": f"Here is a list of the papers and the relevant information: {paper_abstracts}"}
        ]
    )

    gpt_summary = completion.choices[0].message.content
    
    txtfile.write(f"{gpt_summary}\n")
